In [1]:
import pandas as pd  
pd.options.mode.chained_assignment = None
import numpy as np  
from copy import deepcopy
from string import punctuation
from random import shuffle
from sklearn.preprocessing import scale
import advertools as adv


#import gensim
#from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
#LabeledSentence = gensim.models.doc2vec.LabeledSentence # we'll talk about this down below

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()

In [54]:
#data = pd.read_csv("final_df.csv", encoding='utf-8')
#data.iloc[:,1:].to_csv("final_df_.csv", index = False)
data = pd.read_csv("final_df_.csv", encoding='utf-8')

In [2]:
def remove_character(tweet, characters):
    #characters = "'!?"
    tweet = ''.join( x for x in tweet if x not in characters)
    return tweet

def extract_emojis(str): 
    return ''.join(c for c in str if c in emoji.UNICODE_EMOJI)

In [31]:
#text_list = "🤔 🙈 me así, bla es se 😌 ds 💕👭👙"
text_list  =  "🇵🇪 ADELANTO 📺 | «[@KeikoFujimori] está demostrando al #Perú entero y al mundo que no sabe perder y no sabe aceptar su derrota"
emoji_summary = adv.extract_emoji(text_list)
print(emoji_summary['emoji_flat'])

['📺']


In [127]:
#function to retunr specific persons
def clean_tweet(df, label_tweet, characters):
    df['tweet_'] = df.apply(lambda x: remove_character(x[label_tweet], characters), axis=1)
    return df

#A = clean_tweet(data, 'tweet', '@#')
#A[['tweet', 'tweet_']]
def extract_components(tweet):
    subject = []
    emoticons = []
    tweet  = str(tweet).lower()
    #tweet = remove_character(tweet,'@#')
    tokens = tokenizer.tokenize(tweet)
    subjects = list(filter(lambda t: t.startswith('@'), tokens))
    
    emoji = adv.extract_emoji(tweet)
    emoticons = emoji['emoji_flat']
    #print(subjects, emoticons)
    return subjects, emoticons
    
def extract_labels(df, label_tweet):
    #print(df.apply(lambda x: print(extract_components(x[label_tweet])), axis=1))
    df['subject'] = df.apply(lambda x: extract_components(x[label_tweet])[0], axis=1)
    df['emoticons'] = df.apply(lambda x: extract_components(x[label_tweet])[1], axis=1)
    return df
          
df_tweets = extract_labels(data, 'tweet')

In [128]:
#seleccionando las columnas mas importantes
df_tweets = df_tweets[['id', 'date', 'tweet', 'language', 'hashtags', 'user_id', 'username', 'name', 'retweet', 'nlikes', 'nreplies', 'subject', 'emoticons']]

In [136]:
import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def remove_usernames_links(tweet):
    #tweet = re.sub('@[^\s]+','',tweet)
    tweet = re.sub(r"http\S+", "", tweet)
    tweet = remove_emojis(tweet)
    #print(tweet)
    return tweet
#df['tweet'] = df['tweet'].apply(remove_usernames_links)

In [142]:
def pre_clean_tweets(df, label):
    #print(df)
    df['tweets_clean'] = df.apply(lambda x: remove_usernames_links(x[label]), axis=1)
    return df
    #df['tweets_clean'] = df.apply(lambda x: strip_all_entities(x[label]), axis=1)
    
df_tweets = pre_clean_tweets(df_tweets,'tweet')
#df_tweets

In [143]:
df_tweets.to_csv('df_v1.csv', index = False)